In [1]:
import kaggle

In [22]:
! kaggle datasets init -p /app/_data/models/final/ScabHealthy/

Data package template written to: /app/_data/models/final/ScabHealthy/dataset-metadata.json


In [23]:
! kaggle datasets create -p /app/_data/models/final/ScabHealthy/

Starting upload for file eff4_hs__1.h5
100%|█████████████████████████████████████████| 203M/203M [05:07<00:00, 691kB/s]
Upload successful: eff4_hs__1.h5 (203MB)
Starting upload for file eff4_hs__5.h5
100%|█████████████████████████████████████████| 203M/203M [05:07<00:00, 692kB/s]
Upload successful: eff4_hs__5.h5 (203MB)
Starting upload for file eff4_hs__4.h5
100%|█████████████████████████████████████████| 203M/203M [05:17<00:00, 670kB/s]
Upload successful: eff4_hs__4.h5 (203MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Starting upload for file eff4_hs__2.h5
100%|█████████████████████████████████████████| 203M/203M [05:09<00:00, 687kB/s]
Upload successful: eff4_hs__2.h5 (203MB)
Starting upload for file eff4_hs__3.h5
100%|█████████████████████████████████████████| 203M/203M [05:14<00:00, 677kB/s]
Upload successful: eff4_hs__3.h5 (203MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/nataliayurasova/ScabHealthy


0.891
MODEL_BB_PATH= '../input/model-bb-1/bond_box_999_200.h5'
MODEL_PATH = '../input/0865fulltrain/'
IMAGE_SIZE = (380, 380)
DF_PART = '../input/df-kf-plant/df_kf.csv'
PATH = "/kaggle/input/plant-pathology-2021-fgvc8/"
TRAIN_IMG_PATH = PATH+'train_images/'
TEST_IMG_PATH = PATH+'test_images/'
BATCH_SIZE = 32
NUM_CLASSES=6
SEED = 1488
- replace ''-'scab'
https://www.kaggle.com/nataliayurasova/plant-pathology0891/edit


In [1]:
! pip install albumentations
import albumentations

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
import os

import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import tensorflow_addons as tfa
from PIL import Image
from sklearn.model_selection import (
    StratifiedKFold,
    StratifiedShuffleSplit,
    train_test_split,
)
from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB4, EfficientNetB7
from tensorflow.keras.layers import (
    AveragePooling2D,
    AvgPool2D,
    Conv2D,
    Dense,
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import notebook, tqdm

%matplotlib inline

In [3]:
PATH = "/app/_data/"
BATCH_SIZE = 54
SEED = 42
IMAGE_SIZE = 380
NUM_CLASSES = 2
TRAIN_IMG_PATH = "/app/_data/380_npy/"
TEST_IMG_PATH = "/app/_data/test_images/"
feature_columns = [
    "complex",
    "frog_eye_leaf_spot",
    "healthy",
    "powdery_mildew",
    "rust",
    "scab",
]
wrong = [
    "ead085dfac287263.jpg",
    "95276ccd226ad933.jpg",
    "da8770e819d2696d.jpg",
    "cd3a1d64e6806eb5.jpg",
    "ccec54723ff91860.jpg",
]

In [4]:
df_labels = pd.read_csv("../_data/df_csv/labels_21_20.csv", index_col=[0])
df_labels = df_labels.query("image not in @wrong").reset_index(drop=True)
wrong_eff4_ns = pd.read_csv(
    "/app/sandbox/wrong_predictions/eff4/wrong_eff4_ns.csv", index_col=[0]
)
wrong_eff4_ids = pd.read_csv(
    "/app/sandbox/wrong_predictions/eff4/wrong_eff4_ids.csv", index_col=[0]
)

## wrong_eff4_ids

In [5]:
wrong_hsc = wrong_eff4_ids.query(
    "frog_eye_leaf_spot == 0 and frog_eye_leaf_spot_true== 0 and powdery_mildew==0 and powdery_mildew_true==0 and rust==0 and rust_true == 0"
)[["complex", "complex_true", "healthy", "healthy_true", "scab", "scab_true"]]

In [6]:
wrong_hsc.columns

Index(['complex', 'complex_true', 'healthy', 'healthy_true', 'scab',
       'scab_true'],
      dtype='object')

In [7]:
wrong_hsc.query("healthy_true == 0")

,complex,complex_true,healthy,healthy_true,scab,scab_true
28,0,0,8,0,1,9
86,0,0,11,0,0,11
121,0,0,11,0,1,11
126,0,0,7,0,2,8
127,0,0,8,0,0,9
...,...,...,...,...,...,...
1933,0,8,1,0,5,0
1939,0,9,5,0,1,0
2019,0,0,8,0,0,8
2036,0,0,11,0,0,11


In [8]:
for i in range(0, len(wrong_eff4_ids.columns), 2):
    col1 = wrong_eff4_ids.columns[i]
    col2 = wrong_eff4_ids.columns[i + 1]
    print(col1, col2)
    wrong_eff4_ids[(wrong_eff4_ids[col1] != 0) & (wrong_eff4_ids[col2] != 0)][
        [col1, col2]
    ].describe()

complex complex_true


,complex,complex_true
count,872.000000,872.000000
mean,7.176606,11.634174
std,4.290379,2.004674
min,1.000000,5.000000
25%,3.000000,10.000000
50%,7.000000,13.000000
75%,12.000000,13.000000
max,13.000000,13.000000


frog_eye_leaf_spot frog_eye_leaf_spot_true


,frog_eye_leaf_spot,frog_eye_leaf_spot_true
count,1194.000000,1194.000000
mean,8.999162,12.920436
std,3.365796,0.553463
min,1.000000,8.000000
25%,6.000000,13.000000
50%,9.000000,13.000000
75%,12.000000,13.000000
max,13.000000,13.000000


healthy healthy_true


...

,powdery_mildew,powdery_mildew_true
count,98.000000,98.000000
mean,10.418367,12.561224
std,3.522896,1.308705
min,1.000000,8.000000
25%,8.000000,13.000000
50%,12.000000,13.000000
75%,13.000000,13.000000
max,13.000000,13.000000


rust rust_true


,rust,rust_true
count,249.000000,249.000000
mean,8.714859,12.421687
std,4.235390,1.415244
min,1.000000,8.000000
25%,5.000000,13.000000
50%,10.000000,13.000000
75%,13.000000,13.000000
max,13.000000,13.000000


scab scab_true


,scab,scab_true
count,943.000000,943.000000
mean,7.113468,12.787911
std,3.439631,0.869135
min,1.000000,8.000000
25%,4.000000,13.000000
50%,7.000000,13.000000
75%,10.000000,13.000000
max,13.000000,13.000000


## wrong_eff4_ns

In [9]:
df_labels = pd.read_csv("../_data/df_csv/labels_21_20.csv", index_col=[0])

In [45]:
wr_img = wrong_eff4_ns.image.tolist()

In [47]:
df_labels.query("image in @wr_img")["labels"].value_counts()

scab frog_eye_leaf_spot            682
complex                            414
scab frog_eye_leaf_spot complex    200
frog_eye_leaf_spot complex         165
rust frog_eye_leaf_spot            118
scab                               109
rust complex                        91
powdery_mildew complex              87
rust                                49
healthy                             38
frog_eye_leaf_spot                  34
powdery_mildew                      25
Name: labels, dtype: int64

## only scab and healthy

In [10]:
df_labels = df_labels.query('labels=="scab" or labels == "healthy"')
df_labels["image"] = df_labels["image"].str.replace(".jpg", ".npy")
df_labels = df_labels.sample(frac=1, random_state=SEED).reset_index(drop=True)
df_labels.shape

(10520, 8)

In [11]:
feature_columns_hs = ['healthy', 'scab']

In [12]:
# for 380*380
transform = albumentations.Compose(
    [
        albumentations.CLAHE(p=0.1, clip_limit=(1, 2), tile_grid_size=(8, 8)),
        albumentations.OneOf(
            [
                albumentations.MotionBlur((3, 3)),
                albumentations.MedianBlur(blur_limit=3),
                albumentations.GaussianBlur(blur_limit=(3, 3), sigma_limit=0),
                albumentations.Blur(blur_limit=(3, 3)),
            ],
            p=0.2,
        ),
        albumentations.OneOf(
            [
                albumentations.GaussNoise(var_limit=[10, 50], mean=1),
                albumentations.ISONoise(intensity=(0.1, 1), color_shift=(0.01, 0.05)),
                albumentations.ImageCompression(
                    quality_lower=70, quality_upper=100, compression_type=1
                ),
                albumentations.MultiplicativeNoise(
                    multiplier=(0.95, 1.05), per_channel=True, elementwise=True
                ),
                albumentations.Downscale(
                    scale_min=0.6, scale_max=0.99, interpolation=4
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HueSaturationValue(
                    hue_shift_limit=(-7, 7),
                    sat_shift_limit=(-10, 10),
                    val_shift_limit=(-10, 10),
                ),
                albumentations.RandomBrightnessContrast(
                    brightness_limit=0.15,
                    contrast_limit=0.2,
                    brightness_by_max=True,
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.OpticalDistortion(
                    distort_limit=0.05,
                    shift_limit=0.05,
                    border_mode=2,
                ),
                albumentations.ElasticTransform(
                    alpha=2.0,
                    sigma=50.0,
                    alpha_affine=10.0,
                    interpolation=0,
                    border_mode=2,
                ),
                albumentations.GridDistortion(
                    num_steps=5, distort_limit=0.3, interpolation=0, border_mode=2
                ),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.HorizontalFlip(),
                albumentations.VerticalFlip(),
            ],
            p=0.5,
        ),
        albumentations.OneOf(
            [
                albumentations.Rotate(
                    limit=(-180, 180), interpolation=0, border_mode=2
                ),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=180,
                    interpolation=0,
                    border_mode=2,
                ),
            ],
            p=0.5,
        ),
    ]
)

In [13]:
class Generator(keras.utils.Sequence):
    def __init__(
        self,
        df,
        feature_columns,
        images_src_dir,
        batch_size,
        target_image_size,
        shuffle=False,
        augment=True,
        crop=False,
        resize=False,
        normalize=False,
    ):
        self.shuffle = shuffle
        self.batch_size = BATCH_SIZE
        self.df = df
        self.images_dir = images_src_dir
        self.target_image_size = (IMAGE_SIZE, IMAGE_SIZE)
        self.augment = augment
        self.crop = crop
        self.resize = resize
        self.normalize = normalize
        self.feature_columns = feature_columns
        # create label index map
        self.labels = self._read_labels()
        self.n_samples = self.df.shape[0]
        self.n_batches = self.n_samples // self.batch_size
        # shuffle data, also repeated after each epoch if needed
        if self.shuffle:
            np.random.shuffle(self.labels)

    def _read_labels(self):
        """
        Returns list images mapping to 1-hot label
        """

        # label indexes
        label_ixs = self.df[self.feature_columns].values
        image_ixs = self.df["image"].values
        labels = []

        for i in range(len(image_ixs)):
            labels.append([image_ixs[i], label_ixs[i]])
        return labels

    def __len__(self):
        """
        Length in batches
        """
        return self.n_batches

    def __getitem__(self, b_ix):
        """
        Produce batch, by batch index
        """

        assert b_ix < self.n_batches

        b_X = np.zeros(
            (self.batch_size, self.target_image_size[0], self.target_image_size[1], 3),
            dtype=np.uint8,
        )

        b_Y = np.zeros(
            (self.batch_size, self.df[self.feature_columns].shape[1]),
            dtype=np.uint8,
        )

        for i in range(self.batch_size):
            b_X[i], b_Y[i] = self.get_one(
                i + self.batch_size * b_ix,
            )

        return (b_X, b_Y)

    def get_one(self, one_ix):
        """
        Get single item by absolute index
        """
        id = self.labels[one_ix][0]
        src_file = self.images_dir + id

        # read file
        x = np.load(src_file)
        if self.crop:
            coord = self.df[self.df["image"] == id][
                ["x_min", "y_min", "x_max", "y_max"]
            ].values[0]
            orig_hight = x.shape[0]
            orig_width = x.shape[1]
            x_min = coord[0]
            y_min = coord[1]
            x_max = coord[2]
            y_max = coord[3]
            x = x[
                np.int(y_min * orig_hight) : np.int(y_max * orig_hight),
                np.int(x_min * orig_width) : np.int(x_max * orig_width),
            ]

        y = self.labels[one_ix][1]

        # augment
        if self.augment:
            x = self._augment_image(x)

        # normalize (sample-wise)
        if self.normalize:
            x = x.astype(np.float32)
            x = x - np.mean(x, axis=(0, 1))
            x = x / np.std(x, axis=(0, 1))
        return x.astype(np.uint8), y

    def _augment_image(self, x):
        """
        Randomply augment image
        """

        x = transform(image=x)["image"]
        return x

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.labels)

In [14]:
gen = Generator(
    df=df_labels.head(100),
    feature_columns = feature_columns_hs,
    images_src_dir=TRAIN_IMG_PATH,
    target_image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    augment=True,
    crop=False,
    resize=False,
)

In [15]:
a, b = next(iter(gen))

In [16]:
a[0].shape
b[0]

(380, 380, 3)

array([0, 1], dtype=uint8)

In [17]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [18]:
NUM_CLASSES = 2

In [19]:
def get_model():
    inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    base_model = keras.applications.EfficientNetB4(weights=None, include_top=False)
    base_model.load_weights(
        "/app/_data/models/efficientnet-b4_noisy-student_notop.h5",
        by_name=True,
        skip_mismatch=True,
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.0005),
        metrics=[
            "acc",
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="weighted"),
        ],
    )
    return model

In [20]:
# def lr_logger(self, logs: dict):
#     """ Sample logger - adds LR """
#     logs.update({"lr": keras.backend.eval(self.model.optimizer.lr)})


# def gpu_temp_logger(self, logs: dict):
#     stream = os.popen("nvidia-smi -i 0 --query-gpu='temperature.gpu' --format=csv,noheader")
#     output = stream.read()
#     gpu_temp = int(output)
#     logs.update({"gpu_temp": gpu_temp})


# class TensorBoard_Logger(keras.callbacks.TensorBoard):
#     def __init__(self, loggers=[lr_logger], **kwargs):
#         self._loggers = loggers
#         super().__init__(**kwargs)

#     def on_epoch_end(self, epoch, logs=None):
#         logs = logs or {}
#         for logger in self._loggers:
#             logger(self, logs)
#         super().on_epoch_end(epoch, logs)

In [21]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)

for i, (train_index, valid_index) in enumerate(
    skf.split(df_labels["image"], df_labels["labels"])
):
    train, valid = df_labels.loc[train_index], df_labels.loc[valid_index]
    print("\nStarting " + str(i+1) + " skf\n")
    model_name = "eff4_hs__" + str(i + 1) + ".h5"
    log_dir = "logs_hs_" + str(i + 1) + "/"
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor="val_f1_score",
            patience=11,
            restore_best_weights=True,
            verbose=1,
            mode="max",
        ),
        keras.callbacks.ModelCheckpoint(
            "/app/_data/models/final/ScabHealthy/" + model_name,
            monitor="val_f1_score",
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode="max",
            save_freq="epoch",
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_f1_score",
            factor=0.9,
            patience=8,
            verbose=1,
            mode="max",
            min_delta=1e-4,
            min_lr=0.00000001,
        ),
        keras.callbacks.TensorBoard(
            log_dir="/app/.tensorboard/" + log_dir, histogram_freq=0
        ),
        keras.callbacks.experimental.BackupAndRestore(
            "/app/_data/models/final/ScabHealthy/backup/"
        ),
    ]

    gen_train = Generator(
        df=train,
        feature_columns = feature_columns_hs,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=True,
        crop=False,
        resize=False,
    )
    gen_valid = Generator(
        df=valid,
        feature_columns = feature_columns_hs,
        images_src_dir=TRAIN_IMG_PATH,
        target_image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        augment=False,
        crop=False,
        resize=False,
    )
    model = get_model()
    history = model.fit(
        gen_train,
        validation_data=gen_valid,
        epochs=100,
        steps_per_epoch=train.shape[0] // BATCH_SIZE,
        validation_steps=valid.shape[0] // BATCH_SIZE,
        verbose=1,
        workers=15,
        callbacks=callbacks,
    )
    tf.keras.backend.clear_session()


Starting 1 skf

Epoch 1/100
  1/155 [..............................] - ETA: 0s - loss: 0.6968 - acc: 0.5185 - f1_score: 0.5159WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
155/155 [==============================] - ETA: 0s - loss: 0.1309 - acc: 0.9497 - f1_score: 0.9497
Epoch 00001: val_f1_score improved from -inf to 0.80113, saving model to /app/_data/models/final/ScabHealthy/eff4_hs__1.h5
155/155 [==============================] - 77s 495ms/step - loss: 0.1309 - acc: 0.9497 - f1_score: 0.9497 - val_loss: 0.8696 - val_acc: 0.8075 - val_f1_score: 0.8011
Epoch 2/100
155/155 [==============================] - ETA: 0s - loss: 0.0655 - acc: 0.9771 - f1_score: 0.9771
Epoch 00002: val_f1_score improved from 0.80113 to 0.90298, saving model to /app/_data/models/fi